In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=8a1c2829f7269bdd24dadd60c74bd6d882d880a5066dda52df5ae69702dcd86e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
# setup da aplicação Spark
spark = SparkSession \
    .builder \
    .appName("job-glue-spark-tvnews") \
    .getOrCreate()

In [6]:
# definindo o método de logging da aplicação use INFO somente para DEV [INFO,ERROR]
spark.sparkContext.setLogLevel("ERROR")

In [7]:
df_mat = spark.read.format("csv")\
    .option("header", "True")\
    .option("inferSchema","True")\
    .csv("/content/drive/MyDrive/Dados/Tvnews/landing/materia.csv")
df_usu = spark.read.format("csv")\
    .option("header", "True")\
    .option("inferSchema","True")\
    .csv("/content/drive/MyDrive/Dados/Tvnews/landing/usuarios.csv")
df_edi = spark.read.format("csv")\
    .option("header", "True")\
    .option("inferSchema","True")\
    .csv("/content/drive/MyDrive/Dados/Tvnews/landing/editoria.csv")
df_muni = spark.read.format("csv")\
    .option("header", "True")\
    .option("inferSchema","True")\
    .csv("/content/drive/MyDrive/Dados/Tvnews/landing/municipio.csv")
df_tpmat = spark.read.format("csv")\
    .option("header", "True")\
    .option("inferSchema","True")\
    .csv("/content/drive/MyDrive/Dados/Tvnews/landing/tipomateria.csv")

In [ ]:
# imprime os dados lidos da raw
print ("\nImprime os dados lidos da landing:")

# imprime o schema do dataframe
print ("\nImprime o schema do dataframe lido da landing:")

In [8]:
# converte para formato parquet
print ("\nEscrevendo os dados lidos da raw para parquet na processing zone...")
df_mat.write.format("parquet")\
        .mode("overwrite")\
        .save("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_materia.parquet")
df_usu.write.format("parquet")\
        .mode("overwrite")\
        .save("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_usuario.parquet")
df_edi.write.format("parquet")\
        .mode("overwrite")\
        .save("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_editoria.parquet")
df_muni.write.format("parquet")\
        .mode("overwrite")\
        .save("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_municipio.parquet")
df_tpmat.write.format("parquet")\
        .mode("overwrite")\
        .save("/content/drive/MyDrive/Dados/Tvnews/processing2/dob_tipomateria.parquet")



Escrevendo os dados lidos da raw para parquet na processing zone...


In [158]:
# lendo arquivos parquet
df_mat_parquet = spark.read.format("parquet")\
 .load("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_materia.parquet/*.parquet")
df_usu_parquet = spark.read.format("parquet")\
 .load("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_usuario.parquet/*.parquet")
df_edi_parquet = spark.read.format("parquet")\
 .load("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_editoria.parquet/*.parquet")
df_muni_parquet = spark.read.format("parquet")\
 .load("/content/drive/MyDrive/Dados/Tvnews/processing2/dbo_municipio.parquet/*.parquet")
df_tpmat_parquet = spark.read.format("parquet")\
 .load("/content/drive/MyDrive/Dados/Tvnews/processing2/dob_tipomateria.parquet/*.parquet")

In [108]:
df_mat_count = df_mat_parquet.select('CODMATERIA')
df_mat_count.count()

116158

In [159]:
#Selecionando apenas as colunas necessarias dos dataframes
df_usu_parquet = df_usu_parquet.select('CODUSUARIO', 'US_NOME')
df_muni_parquet = df_muni_parquet.select('CODMUNICIPIO', 'MU_NOME')
df_edi_parquet = df_edi_parquet.select('CODEDITORIA', 'ED_DESCRICAO')
df_tpmat_parquet = df_tpmat_parquet.select('CODTIPOMATERIA', 'TM_DESCRICAO')

In [160]:
#Substituindo o codigo dos reportes, produtores, cinegrafistas pelo nome
df_joined = df_mat_parquet.join(df_usu_parquet, df_mat_parquet.CODREPORTER == df_usu_parquet.CODUSUARIO, 'left').withColumnRenamed('US_NOME', 'REPORTER').withColumnRenamed('CODUSUARIO', 'CODREPORTER_')
df_joined = df_joined.join(df_usu_parquet, df_mat_parquet.CODPRODUTOR == df_usu_parquet.CODUSUARIO, 'left').withColumnRenamed('US_NOME', 'PRODUTOR').withColumnRenamed('CODUSUARIO', 'CODPRODUTOR_')
df_joined = df_joined.join(df_usu_parquet, df_mat_parquet.CODCINEGRAFISTA == df_usu_parquet.CODUSUARIO, 'left').withColumnRenamed('US_NOME', 'CINEGRAFISTA').withColumnRenamed('CODUSUARIO', 'CODPRODUTOR_')


In [161]:
#Substituindo o codigo dos municipios, editoria, tipo da materia por seus respectivos nomes
df_joined = df_joined.join(df_muni_parquet, df_mat_parquet.CODMUNICIPIO == df_muni_parquet.CODMUNICIPIO, 'left').withColumnRenamed('MU_NOME', 'MUNICIPIO').withColumnRenamed('CODMUNICIPIO', 'CODREPORTER_')\
                     .join(df_edi_parquet, df_mat_parquet.CODEDITORIA == df_edi_parquet.CODEDITORIA, 'left').withColumnRenamed('ED_DESCRICAO', 'EDITORIA').withColumnRenamed('CODEDITORIA', 'CODEDITORIA_')\
                     .join(df_tpmat_parquet, df_mat_parquet.CODTIPOMATERIA == df_tpmat_parquet.CODTIPOMATERIA, 'left').withColumnRenamed('TM_DESCRICAO', 'TIPO_MATERIA').withColumnRenamed('CODTIPOMATERIA', 'CODTIPOMATERIA_')


In [162]:
df_soft = df_joined.select('CODMATERIA', 'PRODUTOR','REPORTER','CINEGRAFISTA','MUNICIPIO','TIPO_MATERIA','EDITORIA','MA_DATA','MA_LOCAL','MA_RETRANCA')

In [163]:
df_soft = df_soft.withColumn("MA_LOCAL", when(col("MA_LOCAL") == 'POLÍCIA', 'DELEGACIA').otherwise(col("MA_LOCAL")))\
                   .withColumn("MA_LOCAL", when(col("MA_LOCAL") == 'POLICIA', 'DELEGACIA').otherwise(col("MA_LOCAL")))\
                   .withColumn("MA_LOCAL", when(col("MA_LOCAL") == 'ESTÁDIO BARBALHÃO', 'ESTÁDIO COLOSSO DO TAPAJÓS').otherwise(col("MA_LOCAL")))\
                   .withColumn("MA_LOCAL", when(col("MA_LOCAL") == 'COLOSSO DO TAPAJÓS', 'ESTÁDIO COLOSSO DO TAPAJÓS').otherwise(col("MA_LOCAL")))\
                   .withColumn("MA_LOCAL", when(col("MA_LOCAL") == 'CÂMARA', 'CÂMARA DE VEREADORES').otherwise(col("MA_LOCAL")))

In [164]:
#Limpeza na coluna MA_LOCAL
df_final = df_soft.withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('X{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))\
                               .withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('\.{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))\
                               .withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('\,{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))\
                               .withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('\\*,{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))\
                               .withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('Z{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))\
                               .withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('\;{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))\
                               .withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike('-{2,}'), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))

In [165]:
# Crie uma lista das palavras que você deseja substituir
palavras_para_substituir = [ 'NA PAURTA',r'\bS\b',r'\bN PAUTA\b',r'\b-\b','NA PAUTA', r'\b//\b','NA PAUTA//','SANTARÉM','SANTAREM','TV','STM','VÁRIOS','///','/','VER PAUTA','VER PAUTA','INDEFINIDO',r'\bRUA\b','VER NA PAUTA','RUAS','A DEFINIR','\\.', r'\b;\b', r'\bVÍDEO\b','\\*+',r'\bR\b',r'\bRR\b','RRR','SSSS',r'\bD\b','AAA',r'\bAA\b','////','SSS',r'\bSS\b',r'\bQ\b',r'\bVARIOS\b','LLLL','NA ´PAUTA','N APAUTA','NA OPAUTA','GGG','NA PAUTAS']  # note que estamos escapando o ponto

# Crie a expressão regular
regex = '|'.join(palavras_para_substituir)

# Substitua a coluna se ela contiver qualquer uma das palavras na lista
df_final = df_final.withColumn('MA_LOCAL', when(col('MA_LOCAL').rlike(regex), 'NAO ESPECIFICADO').otherwise(col('MA_LOCAL')))

In [ ]:
df_final.show(truncate=False)

In [172]:
df_final.repartition(1)\
          .write\
          .format("parquet")\
          .mode("overwrite")\
          .save("/content/drive/MyDrive/Dados/Tvnews/curated2/tvnews-full.parquet")

116158